In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm, stats
import matplotlib.pyplot as plt
import operator
#import pandas_datareader.data as web
import datetime

### Base de dados atualizada até 03/05/2019

In [28]:
close = pd.read_excel('monthly_close_clean_v3.xlsx', index_col='Date')

In [29]:
close.tail()

,IBOV,QVUM3B,QVQP3B,APPA3,ABCB4,ABYA3,EALT4,AVIL3,ADHM3,AELP3,...,WHRL4,WHMT3,WISA3,WSON33,WIZS3,WLMM4,WWOW3,ILMD4,OPZI3B,ZIVI4
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-31,87887.0,NaN,NaN,NaN,16.236743,NaN,4.29,NaN,1.50,NaN,...,5.10,NaN,NaN,40.000000,7.010000,5.070000,NaN,NaN,NaN,NaN
2019-01-31,97394.0,NaN,NaN,NaN,19.950000,NaN,5.33,NaN,1.41,NaN,...,4.75,NaN,NaN,41.500000,8.050000,6.200000,NaN,NaN,NaN,NaN
2019-02-28,NaN,NaN,0.0,NaN,18.280000,NaN,5.15,NaN,1.67,NaN,...,NaN,NaN,NaN,40.262659,7.858426,6.142275,NaN,NaN,NaN,NaN
2019-03-29,NaN,NaN,0.0,NaN,19.050000,NaN,5.21,NaN,1.60,NaN,...,NaN,NaN,NaN,35.044301,7.430102,6.449869,NaN,NaN,NaN,NaN
2019-04-30,NaN,NaN,0.0,NaN,19.450000,NaN,4.77,NaN,1.52,NaN,...,NaN,NaN,NaN,38.500000,8.190000,5.171430,NaN,NaN,NaN,NaN


In [48]:
close_daily = pd.read_excel('matrix_fix.xlsx', index_col='Data')

In [50]:
close_monthly = close_daily.resample('M').last()
close_monthly.tail()

,Fechamento,QVQP3B,ABCB4,EALT4,ADHM3,TIET11,AFLT3,CANT4B,ALEF3B,BRGE12,...,VLID3,VVAR3,VIVR3,VULC3,WEGE3,MWET4,WHRL3,WSON33,WIZS3,WLMM4
Data,,,,,,,,,,,,,,,,,,,,,
2019-01-31,25.562134,NaN,19.95,5.33,1.41,11.438203,4.50,NaN,NaN,NaN,...,20.89,6.00,3.90,8.18,18.710226,4.00,4.41,39.232222,7.036744,5.959641
2019-02-28,27.041100,NaN,18.28,5.15,1.67,11.428384,4.85,NaN,NaN,3.22,...,19.69,4.58,3.70,7.96,18.373079,4.09,4.64,40.262659,7.858426,6.142275
2019-03-31,28.040401,NaN,19.05,5.21,1.60,10.878565,5.34,NaN,NaN,3.00,...,19.28,4.22,3.70,6.75,18.000000,4.10,4.40,35.044301,7.430102,6.449869
2019-04-30,27.110000,NaN,19.45,4.77,1.52,10.800000,5.02,NaN,NaN,3.01,...,17.75,4.10,2.25,6.18,18.580000,5.00,4.48,38.500000,8.190000,5.171430
2019-05-31,26.730000,NaN,19.29,4.65,1.47,10.800000,5.00,NaN,NaN,2.90,...,17.69,4.14,2.19,6.20,18.770000,5.00,4.50,39.490000,8.280000,5.500000


### Caso queira filtrar pelo Ibovespa:

In [43]:
def planilha_excel_ibov(aba):
    '''
    usar sheet_name ou sheetname
    '''
    df = pd.read_excel('PortfolioCompositionPerQuarter_IBOVESPA_1990_2019.xls', sheetname=aba, skiprows=1)
    df.columns = ['codigo', 'acao', 'tipo', 'qtde', 'part']
    return df.codigo

In [64]:
planilha_excel_ibov('Mai 2019').head()

0    ABEV3
1    AZUL4
2    B3SA3
3    BBAS3
4    BBDC3
Name: codigo, dtype: object

In [5]:
def retorno_ibov_mom(data, mes, mom):
    '''
    retorno_ibov_mom(close2, '2017-12', 6)
    '''
    retorno = data.pct_change(mom)
    rank = retorno[mes].T
    rank = rank.reset_index(drop=False)
    rank.columns = ['Acao', mes]
    rank = rank.sort_values(mes, ascending=False)
    return rank

In [51]:
retorno_ibov_mom(close_monthly, '2019-04', 6).head(15)

,Acao,2019-04
152,GSHP3,220.115930
186,ITEC3,2.805703
74,CEED3,2.124024
69,CASN3,1.500000
62,BPAC11,1.147355
178,INEP4,1.139159
141,BAUH4,1.117687
311,SPRI5,1.060606
189,JBSS3,0.928780
32,BPAN4,0.883333


In [ ]:
def carteira_momentum(data, mes, mom):
    '''
    retorno_ibov_mom(close2, '2017-12', 6)
    '''
    retorno = data.pct_change(mom)
    rank = retorno[mes].T
    rank = rank.reset_index(drop=False)
    rank.columns = ['Acao', mes]
    rank = rank.sort_values(mes, ascending=False)
    return rank

In [ ]:
def portfolio_return(data, mes_ref, mes_analise, qtde_acoes):
    '''
    mes_ref: '2017-12'
    mes_analise: '2018-01'
    '''
    df = retorno_ibov_mom(data, mes_ref, 6).head(qtde_acoes)
    df2 = retorno_ibov_mom(data, mes_analise, 1)
    df3 = pd.merge(df, df2, on='Acao')

    df3.index.name = 'Acao'
    
    df4 = df3.append(df3.agg(['mean']), sort=True)
    return df4

In [98]:
def carteira_mes(aba, tipo_carteira, mes, mom):#, ano_anterior, ano_corrente):
    '''
    Objetivo é selecionar a carteira do mês para identificar quais ações mais subiram
    Pode ser feita análise baseada na carteira do Ibovespa, IBrX, SmallCaps ou todas ações do Fundamentus
    Parâmetros:
    -----------
    aba: 'Jan 2019'
    tipo_carteira: ibov, ibrx, small, fundamentus
    '''    
    if tipo_carteira == 'ibov':

        df = pd.read_excel('PortfolioCompositionPerQuarter_IBOVESPA_1990_2019.xls', sheetname=aba, skiprows=1)
        df.columns = ['codigo', 'acao', 'tipo', 'qtde', 'part']
        df_cod = df.codigo
        df_cod_filter = df_cod.values.tolist()
        carteira1 = close_daily.filter(df_cod_filter, axis=1)
        carteira_mes = carteira1.resample('M').last()
        
        retorno = carteira_mes.pct_change(mom)
        rank = retorno[mes].T
        rank = rank.reset_index(drop=False)
        rank.columns = ['Acao', mes]
        rank = rank.sort_values(mes, ascending=False)
    return rank

In [101]:
def carteira_mes(aba, tipo_carteira, mes, mom):#, ano_anterior, ano_corrente):
    '''
    Objetivo é selecionar a carteira do mês para identificar quais ações mais subiram
    Pode ser feita análise baseada na carteira do Ibovespa, IBrX, SmallCaps ou todas ações do Fundamentus
    Parâmetros:
    -----------
    aba: 'Jan 2019'
    tipo_carteira: ibov, ibrx, small, fundamentus
    '''    
    if tipo_carteira == 'ibov':
        df = pd.read_excel('PortfolioCompositionPerQuarter_IBOVESPA_1990_2019.xls', sheetname=aba, skiprows=1)
        
    elif tipo_carteira == 'ibrx':
        df = pd.read_excel('PortfoliocompositionPerQuarter_IBrX_100_1995_2017.xls', sheetname=aba, skiprows=1)
        
    elif tipo_carteira == 'small':
        df = pd.read_excel('PortfolioCompositionPerQuarter_SMLL_2006_2018.xls', sheetname=aba, skiprows=1)
    
    elif tipo_carteira == 'fundamentus':
        df = pd.read_excel('Fundamentus.xlsx', sheetname=aba, skiprows=1)
    

    df.columns = ['codigo', 'acao', 'tipo', 'qtde', 'part']
    df_cod = df.codigo
    df_cod_filter = df_cod.values.tolist()
    carteira1 = close_daily.filter(df_cod_filter, axis=1)
    carteira_mes = carteira1.resample('M').last()
        
    retorno = carteira_mes.pct_change(mom)
    rank = retorno[mes].T
    rank = rank.reset_index(drop=False)
    rank.columns = ['Acao', mes]
    rank = rank.sort_values(mes, ascending=False)
    return rank

In [102]:
carteira_mes('Mai 2019', 'ibov', '2019-04', 6).head(10)

,Acao,2019-04
35,JBSS3,0.928780
52,SBSP3,0.723567
44,NATU3,0.616092
16,CSNA3,0.519331
15,CSAN3,0.453333
20,EGIE3,0.453301
21,ELET3,0.427197
8,BRFS3,0.417808
53,SMLS3,0.412617
42,MRVE3,0.410245


In [103]:
carteira_mes('Mai 2019', 'ibrx', '2019-04', 6).head(10)

,Acao,2019-04
47,JBSS3,0.928780
83,TOTS3,0.780146
72,SBSP3,0.723567
21,CPLE6,0.643968
63,NATU3,0.616092
73,SAPR11,0.579893
60,MOVI3,0.547579
59,BEEF3,0.530055
77,CSNA3,0.519331
76,SEER3,0.479795


In [89]:
df = pd.read_excel('PortfolioCompositionPerQuarter_IBOVESPA_1990_2019.xls', sheetname='Mai 2019', skiprows=1)
df.columns = ['codigo', 'acao', 'tipo', 'qtde', 'part']
df_cod = df.codigo
df_cod.head()

0    ABEV3
1    AZUL4
2    B3SA3
3    BBAS3
4    BBDC3
Name: codigo, dtype: object

In [97]:
filtro_cod = df_cod.values.tolist()
carteira1 = close_daily.filter(filtro_cod, axis=1)
carteira1
carteira_mes = carteira1.resample('M').last()
carteira_mes    

,ABEV3,AZUL4,B3SA3,BBAS3,BBDC4,BBSE3,BRAP4,BRDT3,BRFS3,BRKM5,...,SBSP3,SMLS3,TAEE11,TIMP3,UGPA3,USIM5,VALE3,VIVT4,VVAR3,WEGE3
Data,,,,,,,,,,,,,,,,,,,,,
2010-01-31,4.353520,NaN,8.764350,15.395705,9.469294,NaN,23.178723,NaN,20.165195,9.385750,...,7.948076,NaN,4.149763,5.757196,NaN,22.884935,32.933161,19.021559,3.048638,4.250223
2010-02-28,4.328270,NaN,8.151603,16.568493,9.426269,NaN,23.542943,NaN,19.696237,9.045995,...,7.694549,NaN,4.167421,6.045055,NaN,23.678526,33.970108,18.024277,2.521430,4.206054
2010-03-31,4.048567,NaN,8.320679,16.727398,9.930158,NaN,26.241754,NaN,21.393418,9.265420,...,8.132782,NaN,4.233641,5.757196,4.777449,28.184555,38.481500,17.617129,3.069268,4.537953
2010-04-30,4.315187,NaN,7.947493,16.805851,9.634659,NaN,23.925076,NaN,20.276852,8.897352,...,8.504770,NaN,4.260129,5.016985,NaN,25.959215,36.008153,16.635441,2.911106,4.239403
2010-05-31,4.320466,NaN,8.524468,14.772716,9.211828,NaN,22.377879,NaN,20.812804,7.637424,...,8.917524,NaN,3.132188,5.765420,NaN,21.235572,33.905982,17.070898,3.002794,4.060274
2010-06-30,4.575647,NaN,8.152005,14.016453,8.571080,NaN,19.831639,NaN,21.220370,8.869039,...,9.554490,NaN,2.957810,6.003933,NaN,22.258026,29.599922,17.662924,2.934028,4.002350
2010-07-31,4.787710,NaN,9.100730,17.365618,10.822116,NaN,22.329494,NaN,22.124698,9.350359,...,8.960838,NaN,3.189578,5.716073,NaN,22.891855,32.922708,18.553409,3.518541,4.398091
2010-08-31,4.898289,NaN,9.082049,16.303333,10.226029,NaN,22.135956,NaN,21.086064,10.957120,...,8.512414,NaN,3.532817,5.748971,NaN,20.370419,31.790248,19.722783,3.438314,4.099098
2010-09-30,5.164021,NaN,10.039922,18.536416,11.392873,NaN,24.494705,NaN,23.055887,12.160422,...,9.656404,NaN,3.667463,5.971034,NaN,21.004248,35.465657,20.305023,3.484158,4.475254


In [ ]:
        retorno = carteira_mes.pct_change(mom)
        rank = retorno[mes].T
        rank = rank.reset_index(drop=False)
        rank.columns = ['Acao', mes]
        rank = rank.sort_values(mes, ascending=False)
    return rank